In [1]:
import pandas as pd
import matplotlib as mpl
from scipy import stats
import scipy.special
from scipy import constants
from colossus.cosmology import cosmology
from astropy.cosmology import z_at_value
params = {'flat': True, 'H0': 67.77, 'Om0': 0.307, 'Ob0': 0.04825, 'sigma8': 0.8288, 'ns': 0.9611}
cosmology.addCosmology('planck14', params)
cosmo = cosmology.setCosmology('planck14')

In [2]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import h5py
import matplotlib as mpl
from matplotlib import gridspec
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator, FixedLocator
from matplotlib.backends.backend_pdf import PdfPages

plt.style.use("shao.mplstyle")
fontSize = 15
lineWidth = 1.5

colors = [u'#1f77b4', u'#ff7f0e', u'#2ca02c', u'#d62728', u'#9467bd', u'#8c564b', u'#e377c2', u'#7f7f7f', \
          u'#bcbd22', u'#17becf']

# functions

In [7]:
def read_galform(file_num, simulation_name, maxn=1000, mass_min=1e2,r = 300):
    # print ("file = ", inputRoot + '/' + inputFile) 
    fi = 'M' + str(file_num) + '_'+ simulation_name
    inputRoot = '/home/zhaox/Magpie/Galform/'+fi+'/galaxies.hdf5'
    inputFile = 'Output001'
    h = 0.6776999831199646 
    if simulation_name == 'zcut7':
        if file_num == 4:
            centerIndex = 518
        elif file_num == 5:
            centerIndex = 79
        else:
            centerIndex = 0
    elif simulation_name == '7DM_GAS':
        if file_num == 2:
            centerIndex = 387
        elif file_num == 4:
            centerIndex = 581
        else:
            centerIndex = 0
    # get the Galaxy num
    with h5py.File(inputRoot, 'r' ) as hf:
        data = hf[inputFile]
        # print ("data.keys = " , data.keys())
        gid = data['GalaxyID'][:]
        num = len(gid)
        

        #read data
        mass =  data['mstars_bulge'][:]/h + data[ 'mstars_disk'][:]/h
        Mhalo = data['mhhalo'][:]/h
        # 坐标单位kpc(n,3),速度单位km/s(n,3)
        pos = np.array([data[ 'xgal'][:], data['ygal'][:], data['zgal'][:]]).T *1000/h
        vel = np.array([data['vxgal'][:], data['vygal'][:], data['vzgal'][:]]).T
        center = pos[centerIndex]
        pos = pos - center
        vel = vel - vel[centerIndex]
        print('center pos:', center*h/1000)
    index = []
    dis = []
    for i in range(num):
        dis_ = ((pos[i][0]**2 + pos[i][1]**2 + pos[i][2]**2)**0.5)
        # 筛选条件小于300kpc，大于100msun
        if  dis_< r and mass[i] > mass_min:
            index.append(i)
            dis.append(dis_)
            
    index = np.array(index)
    print("Galform Mhalo=%e"%Mhalo[centerIndex])
    # 放入pandas DataFrame中，按质量排序
    satellite_data =pd.DataFrame({"id":index, \
        "mass":mass[index], "distance[kpc]":dis,\
        "x[kpc]":pos[index][:,0],"y[kpc]":pos[index][:,1],"z[kpc]":pos[index][:,2],\
        "vx[km/s]":vel[index][:,0],"vy[km/s]":vel[index][:,1],"vz[km/s]":vel[index][:,2]},  \
    columns=['id', 'mass', 'distance[kpc]', 'x[kpc]', 'y[kpc]', 'z[kpc]', 'vx[km/s]', 'vy[km/s]', 'vz[km/s]'])
    satellite_data = satellite_data.sort_values(by=['mass'],ascending=False)
    # 返回显示质量最大的maxn个,maxn=1000表示输出所有满足条件的星系，从1开始是因为0是中央星系
    print ("Galaxy num = " , num, "satellite num = ", len(index)-1)
    satellite_pos = np.array(satellite_data[['x[kpc]','y[kpc]','z[kpc]']])
    satellite_vel = np.array(satellite_data[['vx[km/s]','vy[km/s]','vz[km/s]']])
    return Mhalo[centerIndex], satellite_data.iloc[:maxn+1], satellite_pos[1:maxn+1], satellite_vel[1:maxn+1]


def read_hydro(file_num, simulation, maxn):
    # read the subgroup catalogue
    # inputFile_root = "groups_199_z000p000/eagle_subfind_tab_028_z000p000.%i.hdf5" 
    file = simulation_name
    inputFile_root = '/home/zhaox/Magpie/' + file + '/groups_199_z000p000/eagle_subfind_tab_199_z000p000.%i.hdf5'
    # read the data - read the first tab file to get the total number of subgroups
    inputFile = inputFile_root % 0

    # 对于4和5，中央星系的galaxyID不等于0

    if simulation == 'zcut7':
        if file_num == 4:
            centerIndex = 2
        elif file_num == 5:
            centerIndex = 2
        else:
            centerIndex = 1
    elif simulation == '7DM_GAS':
        if file_num == 2:
            centerIndex = 2
        elif file_num == 4:
            centerIndex = 3
        else:
            centerIndex = 1
    else:
        print("can't understand simulation_name!")
        return
    with h5py.File( inputFile, 'r' ) as hf:
        header  = hf['Header'].attrs
        # print ('header keys:', header.keys())
        numFiles = header['NTask']  # this gives the number of files in which the group catalog is saved
        noTotGrps = header['TotNgroups']  # total number of FOF groups
        noTotSubs = header['TotNsubgroups']  # total number of SUBFIND subhaloes
        hFactor   = header['HubbleParam']
        # print("All groups in the file:", hf.keys())
        # print("All entries in the 'FOF' group:", hf['FOF'].keys())
        # print("All entries in the 'SubHalo' group:", hf['Subhalo'].keys())
        # print(numFiles, noTotGrps, noTotSubs, hFactor)


    # reserve memory for the output arrays
    FOF_length   = np.zeros( noTotGrps, np.uint32 )
    FOF_mass     = np.zeros( noTotGrps, np.float32 )
    FOF_pos      = np.zeros( (noTotGrps,3), np.float32 )  # centre of potential
    FOF_M200     = np.zeros( noTotGrps, np.float32 )
    FOF_R200     = np.zeros( noTotGrps, np.float32 )
    Subhalo_mass = np.zeros( noTotSubs, np.float32 )
    Subhalo_pos  = np.zeros( (noTotSubs,3), np.float32 )  # centre of potential
    Subhalo_vel = np.zeros( (noTotSubs,3), np.float32 )
    Subhalo_gnum = np.zeros( noTotSubs, np.float32 )

    # loop over all the various files
    start, end = 0, 0
    start2, end2 = 0, 0
    for i in range(numFiles):
        inFile = inputFile_root % i
        with h5py.File( inFile, 'r' ) as hf:
            h = hf['Header'].attrs
            numGrps = h['Ngroups']  # number of groups this file
            numsubGrps = h['Nsubgroups']
            end = start + numGrps
            end2 = start2 + numsubGrps
    #         print(numsubGrps)
            # print ("Reading file %i of %i --%s-- which contains %i groups ..." % ( i+1, numFiles, inFile, numGrps ))
            
            # FOF_length[start:end] = hf["FOF/GroupLength"]
            # FOF_mass[start:end] = hf["FOF/GroupMass"]
            FOF_pos[start:end]  = hf["FOF/GroupCentreOfPotential"]
            FOF_M200[start:end] = hf["FOF/Group_M_Crit200"]
            FOF_R200[start:end] = hf["FOF/Group_R_Crit200"]
            Subhalo_pos[start2:end2] = hf["Subhalo/CentreOfPotential"]
            Subhalo_mass[start2:end2] = hf["Subhalo/MassType"][:,4]
            Subhalo_vel[start2:end2] = hf["Subhalo/Velocity"]
            Subhalo_gnum[start2:end2] = hf["Subhalo/GroupNumber"]
            start = end
            start2 = end2
    Subhalo_pos = (Subhalo_pos - FOF_pos[centerIndex-1])* 1000/hFactor
    satellite_index = []
    dis = []
    for i in range(len(Subhalo_pos)):
        dis_ = ((Subhalo_pos[i][0]**2 + Subhalo_pos[i][1]**2 + Subhalo_pos[i][2]**2)**0.5)
        # 筛选条件小于300kpc，大于100msun
        # print(dis_)
        if  dis_  < 300 and Subhalo_mass[i] > 0:
            satellite_index.append(i)
            dis.append(dis_)
    # print(satellite_index)
    # 选择groupNumber==centerindex的星系
    # satellite_index = np.where(Subhalo_gnum == centerIndex)

    # print(satellite_index)
    print('r200 = ',FOF_R200[centerIndex-1])
    #计算相对坐标和相对速度
    # satellite_pos = (Subhalo_pos[satellite_index] - FOF_pos[centerIndex-1])* 1000/hFactor
    satellite_pos = Subhalo_pos[satellite_index]
    satellite_vel = Subhalo_vel[satellite_index] - Subhalo_vel[satellite_index][0]
    dis = np.linalg.norm(satellite_pos, axis = 1)
    satellite_data =pd.DataFrame({"groupNumber":Subhalo_gnum[satellite_index], "id":satellite_index,\
            "mass":Subhalo_mass[satellite_index]*1e10/hFactor, "distance[kpc]":dis, \
            "x[kpc]":satellite_pos[:,0],"y[kpc]":satellite_pos[:,1],"z[kpc]":satellite_pos[:,2],\
            "vx[km/s]":satellite_vel[:,0],"vy[km/s]":satellite_vel[:,1],"vz[km/s]":satellite_vel[:,2]},  \
        columns=['groupNumber', 'id', 'mass', 'distance[kpc]', 'x[kpc]', 'y[kpc]', 'z[kpc]', 'vx[km/s]', 'vy[km/s]', 'vz[km/s]'])
        # 返回显示质量最大的maxn个,maxn=1000表示输出所有满足条件的星系，从1开始是因为0是中央星系
    # satellite_data = all_data[all_data['groupNumber'] == centerIndex+1] 
    # satellite_data = satellite_data.query("mass > 0 & `distance[kpc]` < 300")
    # satellite_index = np.where(satellite_data['groupNumber'] == centerIndex)
    # satellite_data = satellite_data.iloc[satellite_index]
    satellite_data = satellite_data.sort_values(by=['mass'],ascending=False)
    # satellite_data = satellite_data.loc[satellite_data['distance[kpc]']<300 ]
    satellite_pos = np.array(satellite_data[['x[kpc]','y[kpc]','z[kpc]']])
    satellite_vel = np.array(satellite_data[['vx[km/s]','vy[km/s]','vz[km/s]']])

    print('center pos:', FOF_pos[centerIndex-1], "satellite num = ", len(satellite_pos)-1)
    return FOF_R200[centerIndex-1], FOF_M200[centerIndex-1], satellite_data.iloc[:maxn+1], satellite_pos[1:maxn+1], satellite_vel[1:maxn+1]
    # return satellite_data.sort_values(by=['mass'],ascending=False).iloc[:]

In [4]:
def pos_match(hydro_pos, galform_pos):
    match = np.full(len(hydro_pos), -1,dtype=int)
    for i in range(len(hydro_pos)):
        for j in range(len(galform_pos)):
            if np.linalg.norm(hydro_pos[i] - galform_pos[j]) < 0.1:
                match[i] = j
                continue
    return match

In [5]:
def read_data(file, mass_min = 100, r_min = 300, after_destruction=True):
# within 300kpc, > mass_min, after tidal destruction

    with h5py.File(file, 'r' ) as hf:
        # data = hf[inputFile]
        # print ("data.keys = " , data.keys())
        # gid = data['GalaxyID'][:]
        # num = len(gid)
        print("hf.keys()", hf.keys())
        # print("hf/satellite_galaxies.keys()",hf['satellite_galaxies'].keys())
        t = hf["header_info/time"][:]
        a = hf["header_info/scale_factor"][:]
        central_Mhalo = hf['central_galaxy/Mhalo'][:]
        satellite_mstar = hf['satellite_galaxies/Mstar'][:]
        satellite_msub = hf['satellite_galaxies/Msub'][:]
        # satellite_mhalo = hf['satellite_galaxies/Mhalo'][:]
        central_pos  = hf['central_galaxy/pos'][:]
        satellite_pos = hf['satellite_galaxies/pos'][:]
        satellite_vel = hf['satellite_galaxies/vel'][:]
        # is_MW_central = hf['satellite_galaxies/is_MW_central'][:]
        # Last snapshot at which the galaxy is a FOF central subhalo. Different from Galform definition of central galaxy.
        # satellite_snap_last_FOF_central = hf['satellite_galaxies/snap_last_FOF_central'][:]
        # Last snapshot at which the galaxy was a central.
        # satellite_snap_last_central = hf["satellite_galaxies/snap_last_central"][:]
        # satellite_type = hf["satellite_galaxies/type"][:]
        destruction = hf["tidal_disruption/destroyed_orbit_interpolation"][:]
    print("d<300,m>0 satellite num=", len(satellite_mstar))
    print("d<300,m>100 satellite num=", sum(satellite_mstar[:,0]>100))
    
    # R200 = 227.2 * (central_Mhalo/1.252e12)**0.3333
    is_satellite = np.logical_and(satellite_mstar[:,0]>mass_min, destruction==False)
    if after_destruction:
        satellite_mstar = satellite_mstar[is_satellite]
        satellite_pos = satellite_pos[is_satellite]
        satellite_vel = satellite_vel[is_satellite]
    print("d<300,m>100 after destruction satellite num=", sum(is_satellite))
    return satellite_msub, satellite_mstar, satellite_pos, satellite_vel, central_pos,central_Mhalo

In [6]:
h = 0.6776999831199646

In [5]:
simulation_num = 1
simulation = 'zcut7'
# simulation = '7DM_GAS'
simulation_name = 'M' + str(simulation_num) + '_' + simulation
if simulation == '7DM_GAS':
    file = "/home/zhaox/Magpie/Destruction/satellite_orbits_Galform/M" + str(simulation_num) + "_7DM_GAS.hdf5"
elif simulation =='zcut7':
    file = "/home/zhaox/Magpie/Destruction/satellite_orbits_Galform/M" + str(simulation_num) + "_MR.hdf5"
satellite_msub, satellite_mstar, satellite_pos, satellite_vel, central_pos,central_Mhalo = read_data(file, after_destruction=False)
sort = np.argsort(satellite_mstar[:,0])[::-1] # 大到小
satellite_pos = satellite_pos[sort]
satellite_vel = satellite_vel[sort]
satellite_mstar = satellite_mstar[sort]
satellite_msub = satellite_msub[sort]
print("central_Mhalo=%e"%central_Mhalo[0])
print("satellite_msub=",satellite_msub[:,0])

NameError: name 'read_data' is not defined

In [8]:
central_pos[0]/1000*h

array([27.89449692, 38.13427734, 25.12496567])

In [8]:

maxn = 1000
file_num = simulation_num
h = cosmo.h
# r = 0.13329083*1000/h
r = 350

simulation_name = 'M' + str(file_num)+'_' +simulation
# simulation_name = '7DM_GAS'
hydro_R200, hydro_M200, hydro_satellite_data, hydro_pos, hydro_vel = read_hydro(file_num, simulation,maxn)
# galform_satellite_data, galform_pos, galform_vel = read_galform(file_num, simulation,maxn, r=r)

r200 =  0.13329083
center pos: [47.606144 49.335148 20.3374  ] satellite num =  24


In [9]:
hydro_R200

0.13329083

In [11]:
for i in range(1,6,1):
    print(i)

1
2
3
4
5


In [12]:
central_R200 = []
for i in range(1,6,1):
    file_num = i
    simulation = 'zcut7'
    simulation_name = 'M' + str(file_num)+'_' +simulation
    hydro_R200, hydro_M200, hydro_satellite_data, hydro_pos, hydro_vel = read_hydro(file_num, simulation,maxn)
    central_R200.append(hydro_R200)
    simulation = '7DM_GAS'
    simulation_name = 'M' + str(file_num)+'_' +simulation
    hydro_R200, hydro_M200, hydro_satellite_data, hydro_pos, hydro_vel = read_hydro(file_num, simulation,maxn)
    central_R200.append(hydro_R200)

r200 =  0.13329083
center pos: [47.606144 49.335148 20.3374  ] satellite num =  24
r200 =  0.13497305
center pos: [47.57527  49.336098 20.316504] satellite num =  31
r200 =  0.1473333
center pos: [26.079184 31.32167  42.181503] satellite num =  30
r200 =  0.14724852
center pos: [26.076216 31.32041  42.159283] satellite num =  29
r200 =  0.15836081
center pos: [48.15624  33.671974 47.70441 ] satellite num =  29
r200 =  0.15805022
center pos: [48.16409  33.67621  47.680977] satellite num =  32
r200 =  0.13992597
center pos: [27.894497 38.134277 25.124966] satellite num =  32
r200 =  0.1446742
center pos: [27.876484 38.108208 25.096678] satellite num =  29
r200 =  0.100427255
center pos: [38.66934 22.42929 45.06645] satellite num =  11
r200 =  0.099346854
center pos: [38.662483 22.43605  45.048016] satellite num =  14


In [13]:
np.savetxt('/home/zhaox/app/after_destruction/central_R200.txt', np.array(central_R200))

In [14]:
central_R200

[0.13329083,
 0.13497305,
 0.1473333,
 0.14724852,
 0.15836081,
 0.15805022,
 0.13992597,
 0.1446742,
 0.100427255,
 0.099346854]

In [10]:
galform_Mhhalo, galform_satellite_data, galform_pos, galform_vel = read_galform(file_num, simulation,maxn, r=r)

center pos: [27.894497 38.134277 25.124966]
Galform Mhalo=1.098704e+12
Galaxy num =  1212 satellite num =  145


In [11]:
pos_match(hydro_pos, galform_pos)

array([ 0,  2,  1,  8, 12, 10,  7,  6, 19, 15, 28, 11, 34, 20, 23, 32, 84,
       39, 52, 33, 75, 30, 63, 49, 41, 85, 66, 67, 83, 42, 31, 89])

In [12]:
print("before destruction:")
pos_match(hydro_pos, satellite_pos[:,0])

before destruction:


array([ 0,  2,  1,  8, 12, 10,  7,  6, 18, 15, 26, 11, 32, 19, 22, 30, 81,
       36, 49, 31, 72, 28, 60, 46, 38, 82, 63, 64, 80, 39, 29, 86])

In [21]:
satellite_msub, satellite_mstar, satellite_pos, satellite_vel, central_pos,central_Mhalo = read_data(file, after_destruction=True)
sort = np.argsort(satellite_mstar[:,0])[::-1] # 大到小
satellite_pos = satellite_pos[sort]
satellite_vel = satellite_vel[sort]
satellite_mstar = satellite_mstar[sort]
print("after destruction")

pos_match(hydro_pos, satellite_pos[:,0])

hf.keys() <KeysViewHDF5 ['central_galaxy', 'header_info', 'metadata', 'satellite_galaxies', 'tidal_disruption']>
d<300,m>0 satellite num= 149
d<300,m>100 satellite num= 142
d<300,m>100 after destruction satellite num= 93
after destruction


array([ 0,  2,  1,  5,  9,  7,  4,  3, 11, 10, 17,  8, 23, 12, 14, 21, 52,
       25, 31, 22, 47, 19, 40, 29, 26, 53, 41, 42, 51, -1, 20, 56])

In [14]:
print(simulation_name+":")
# print("star mass--------------")
# print("Galform star mass=%e:"%(galform_satellite_data.loc[0]['mass']))
# print("hydro massType[:,4]=%e:"%(hydro_satellite_data.loc[0]['mass']))
print("halo mass--------------")
print("Hydro M200=%e"%(hydro_M200 * 10**10/h) )
print("Galform central mchalo[0]=%e"%(central_Mhalo[0]))
print("Galform mhhalo=%e"%galform_Mhhalo)

M4_zcut7:
halo mass--------------
Hydro M200=9.399319e+11
Galform central mchalo[0]=1.098704e+12
Galform mhhalo=1.098704e+12


In [15]:
galform_satellite_data

,id,mass,distance[kpc],x[kpc],y[kpc],z[kpc],vx[km/s],vy[km/s],vz[km/s]
0,518,1.762396e+10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,526,2.000817e+09,72.785532,46.324219,54.523438,13.378906,41.865532,-85.956444,17.960556
138,668,5.498566e+07,263.879453,-224.996094,135.566406,-25.117188,2.563797,-93.290306,103.900299
3,522,3.277911e+07,265.305294,-139.332031,-193.855469,115.730469,41.274109,-47.771118,59.093048
47,571,2.266948e+07,5.766055,2.843750,1.722656,-4.710938,64.780617,284.278381,-53.185791
...,...,...,...,...,...,...,...,...,...
94,619,1.529018e+02,34.071756,-27.257812,18.113281,9.476562,-38.814651,-74.841469,72.707275
23,544,1.479329e+02,174.047644,-48.023438,36.144531,-163.339844,108.650002,33.424301,81.643982
130,657,1.454682e+02,20.647183,-13.628906,-15.484375,-0.890625,31.838280,-6.894653,-0.128601
37,561,1.403359e+02,151.285999,-22.628906,149.359375,-8.195312,-58.291443,23.287994,-24.920792


In [16]:
satellite_pos.shape

(149, 199, 3)

In [17]:
dis_l = np.zeros((len(satellite_pos)-1,199))
for i in range(len(satellite_pos)-1):
    for j in range(199):
        dis_l[i][j] = np.linalg.norm(satellite_pos[i+1][j])

In [18]:
parameter = np.loadtxt("/home/zhaox/Magpie/Galform/M1_7DM_GAS/output.times")
t = list(reversed(parameter[:,3]))
lookbackTime = cosmo.age(0) - t

In [19]:
# fig, ax = plt.subplots(figsize = (10,8))
# # ax.plot(lookbackTime, dis_l[0], label = 'lmc')
# # ax.plot(lookbackTime, dis_l[1], label = 'smc')
# index = np.arange(0,11,1)

# # index = latter_index
# # index = original_index
# # ax.plot(lookbackTime, R200, label = 'r200', linewidth = 5)
# # ax.semilogy(lookbackTime, R200, label = 'r200', linewidth = 5)
# ax.plot(lookbackTime, dis_l[0], label = '0')
# ax.plot(lookbackTime, dis_l[1], label = '1')
# ax.plot(lookbackTime, dis_l[2], label = '2')
# ax.plot(lookbackTime, dis_l[3], label = '3')
# ax.plot(lookbackTime, dis_l[4], label = '4')
# ax.plot(lookbackTime, dis_l[5], label = '5')
# # for i in range(satellite_num_r200):
# # for i in born_infall_galaxy:
#     # if bring_by_lmc[i] == True:
#     #     ax.plot(lookbackTime, dis_l[i], color = 'black')
#     # elif bring_by_smc[i] == True:
#     #     ax.plot(lookbackTime, dis_l[i], color = 'red')
#     # else:
#     # ax.semilogy(lookbackTime, dis_l[i])
#     # ax.plot(lookbackTime, dis_l[i])
# ax.set_xlim(14,0)
# ax.set_ybound(lower=0)
# # ax.set_ylim(bottom=0)
# # ax.set_ylim([1.e1, 1.e3])
# ax.set_xlabel('lookback time/[Gyr]')
# ax.set_ylabel('D/[kpc]')
# ax.set_ylim(0,650)
# plt.legend()
# def lbtime2z(lbtime):
#     t_universe = cosmo.age(0)
#     z = cosmo.age(t_universe - lbtime, inverse = True)
#     return ["%.1f" % z_ for z_ in z]


# title = 'satellite distance evolution'
# ax2 = ax.twiny()
# ax2.set_xlim(ax.get_xlim())
# new_tick_locations = np.array([0.01, cosmo.lookbackTime(0.2), cosmo.lookbackTime(0.5), cosmo.lookbackTime(1), cosmo.lookbackTime(2), cosmo.lookbackTime(3), cosmo.lookbackTime(5)])
# ax2.minorticks_off()
# ax2.set_xticks(new_tick_locations)
# ax2.set_xticklabels(lbtime2z(new_tick_locations))
# ax2.set_xlabel('z')

# plt.suptitle(title)